# Make efficiency map plots for trained fitters

In [3]:
import train as tn
reload(tn)

import plotting
reload(plotting)

import util as ut

import matplotlib.pyplot as plt
#plt.style.use('seaborn-ticks')
plt.style.use('seaborn-poster')
%matplotlib inline
#%matplotlib notebook

import numpy as np

from pprint import pprint

import itertools

# import xgboost

load the fitter:

In [4]:
path,name = "./classifiers", "effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out"
fitter = tn.IO.load(name,path,nodata=True)

./classifiers
./classifiers/effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out.pkl.gz
loading pickle ./classifiers/effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out.pkl.gz


In [5]:
def GetfeatureGrid (branches, projection) :
    feature_grid = []
    
    for bran in branches :
        if 'Pt' in bran :
            feature_grid.append((1,0.,300.))
        if 'Rapidity' in bran :
            feature_grid.append((1,-3.,3.))
            if 'abs' in bran :
                feature_grid[branches.index(bran)] = (1,0.,3.)
                
    for proj in projection :
        if 'Pt' in proj :
            feature_grid[branches.index(proj)] = (300,0.,300.)
        if 'Rapidity' in proj :
            feature_grid[branches.index(proj)] = (300,-3.,3.)
            if 'abs' in proj :
                feature_grid[branches.index(proj)] = (300,0.,3.)
           
    
    return feature_grid

In [6]:
def plot_efficiency_map_Njets (fitter, feedClass = True, projection = ['genPt','absGenRapidity']):
    """
    This function is still in the developpment phase
    So far it is used to produce efficiency plots of Pt vs |y|
    : params 
            fitter : train.EffFitter - the trained classifier
         feedClass : boolean - has influence only on recoPtGrid which is never used
    """
    
    
    
    
    #generate 300 pt bins from 0-300 and 300 pseudo-rap bins from 0-3
    class_grid,class_eff_map = fitter.effMap('class',[(300,0,300),(300,0,3.)])
   

    branches = ['genJet2p5Pt0', 'genJet2p5Rapidity0', 'genJet2p5Pt1', 'genJet2p5Rapidity1', 'genJet2p5Pt2', 
            'genJet2p5Rapidity2', 'genJet2p5Pt3', 'genJet2p5Rapidity3', 'genPt', 'absGenRapidity']
    
    

    feature_grid = GetfeatureGrid(branches,projection)    
    axes = map(tn.mk_grid_1d,feature_grid)        
    feature_grid = np.array(list(itertools.product(*axes)))
    
    
    recoNjet2p5Grid =  np.hstack([feature_grid,class_eff_map]) if feedClass else class_grid
   
    
    recoNjets2p5_grid,recoNjets2p5_eff_map = fitter.effMap('recoNjets2p5',recoNjet2p5Grid)
    
    
    x_index = branches.index(projection[0])
    y_index = branches.index(projection[1])
    
    print(np.shape(recoNjets2p5_grid[:,x_index]))
    print(np.shape(recoNjets2p5_eff_map))
    
    plotting.efficiency_map(recoNjets2p5_grid[:,x_index],recoNjets2p5_grid[:,y_index],recoNjets2p5_eff_map,
                            figsize=(14,26),xlabel=projection[0],ylabel=projection[1],
                        layout=(3,None))
    
    plt.subplots_adjust(hspace = 0.35, wspace = 0.4, top=0.88)



In [ ]:
reload(plotting)
plot_efficiency_map_Njets(fitter, projection = ['genJet2p5Pt0','genJet2p5Rapidity0'])

# Projection Plots using the data frame

In [ ]:
#note that the loading option is True by default
ut.defaultParameters(dataDir='./data', inputName="effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out", 
                     inputDir='./classifiers')
ut.setParams()

In [ ]:
effFitter = ut.loadOrMake()

In [ ]:
df = effFitter.df

In [ ]:
first_train_evt = int(round(df.index.size*(1.-effFitter.split_frac)))
df_test = df[:first_train_evt]

In [ ]:
projection = ['genJet2p5Pt0','genJet2p5Rapidity0']

In [ ]:
def GetNames (list_name,keyword) :
    listNames = []
    for n in list_name :
        if keyword in n :
            listNames.append(n)
    return listNames

In [ ]:
probaNames = GetNames(df.columns,'recoNjets2p5Cat_')

In [ ]:
extract_col = np.concatenate([projection,probaNames])

In [ ]:
df_test.shape

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999

In [ ]:
df_test[extract_col][0:5].T

In [ ]:
X = df_test[projection[0]]
Y = df_test[projection[1]]
Z = df_test[probaNames]

xi, yi = np.linspace(0., 300., 300), np.linspace(-3., 3., 300)
xi, yi = np.meshgrid(xi, yi)

In [ ]:
print(np.shape(X))
print(np.shape(Z))

In [ ]:
 plotting.efficiency_map(X,Y,Z, figsize=(14,26),xlabel=projection[0],ylabel=projection[1],
                        layout=(3,None))

In [ ]:
import scipy.interpolate


In [ ]:

# Set up a regular grid of interpolation points
xi, yi = np.linspace(0., 300., 300), np.linspace(0., 2., 300)
xi, yi = np.meshgrid(xi, yi)


# Interpolate; there's also method='cubic' for 2-D data such as here
zi = scipy.interpolate.griddata((X, Y), Z, (xi, yi), method='linear')

print(np.shape(zi))
plt.imshow(zi, vmin=Z.min(), vmax=Z.max(), origin='lower')#, extent=[0., 300, 0, 3])

#plt.xticks(np.arange(0,300,step=50))
#plt.yticks(np.arange(0,3,step=0.5))
plt.colorbar()
plt.show()


In [ ]:
#fig = plt.figure()
#cbar_ax = fig.add_axes([0.15, 0.97, 0.7, 0.02])
#plt.colorbar(orientation='horizontal')
plt.hexbin(x=X,y=Y,C=Z,cmap=plt.cm.viridis,vmin=0,vmax=1)